## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Flinders,AU,-34.5833,150.8552,68.25,94,100,5.99,moderate rain
1,1,Carnarvon,AU,-24.8667,113.6333,73.47,94,53,10.36,broken clouds
2,2,Vaini,TO,-21.2000,-175.2000,82.56,83,75,13.80,broken clouds
3,3,Busselton,AU,-33.6500,115.3333,63.61,80,99,16.13,overcast clouds
4,4,Baculin,PH,7.4525,126.5842,79.00,86,41,3.85,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.2000,-175.2000,82.56,83,75,13.80,broken clouds
4,4,Baculin,PH,7.4525,126.5842,79.00,86,41,3.85,scattered clouds
5,5,Bara,NG,10.3744,10.7288,75.43,12,3,9.40,clear sky
15,15,Cobija,BO,-11.0267,-68.7692,77.13,86,80,2.75,broken clouds
18,18,Pizarro,CO,4.9533,-77.3660,78.87,85,100,3.42,overcast clouds
23,23,Rikitea,PF,-23.1203,-134.9692,78.46,69,78,3.83,broken clouds
29,29,Hilo,US,19.7297,-155.0900,82.13,65,100,13.80,overcast clouds
33,33,Mount Isa,AU,-20.7333,139.5000,80.37,26,3,11.50,clear sky
36,36,West Bay,KY,19.3667,-81.4167,80.67,64,20,16.11,few clouds
49,49,Luganville,VU,-15.5333,167.1667,85.51,68,90,0.65,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
4,Baculin,PH,79.00,scattered clouds,7.4525,126.5842,
5,Bara,NG,75.43,clear sky,10.3744,10.7288,
15,Cobija,BO,77.13,broken clouds,-11.0267,-68.7692,
18,Pizarro,CO,78.87,overcast clouds,4.9533,-77.3660,
23,Rikitea,PF,78.46,broken clouds,-23.1203,-134.9692,
29,Hilo,US,82.13,overcast clouds,19.7297,-155.0900,
33,Mount Isa,AU,80.37,clear sky,-20.7333,139.5000,
36,West Bay,KY,80.67,few clouds,19.3667,-81.4167,
49,Luganville,VU,85.51,overcast clouds,-15.5333,167.1667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping") 

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace = True)
clean_hotel_df = hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Baculin,PH,79.00,scattered clouds,7.4525,126.5842,Baculin Cove Beach Resort
15,Cobija,BO,77.13,broken clouds,-11.0267,-68.7692,Hotel Obelisco
18,Pizarro,CO,78.87,overcast clouds,4.9533,-77.3660,Hotel Limonar
23,Rikitea,PF,78.46,broken clouds,-23.1203,-134.9692,People ThankYou
29,Hilo,US,82.13,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Mount Isa,AU,80.37,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
36,West Bay,KY,80.67,few clouds,19.3667,-81.4167,Calypso Cove
49,Luganville,VU,85.51,overcast clouds,-15.5333,167.1667,The Espiritu Hotel
51,Umm Lajj,SA,76.15,broken clouds,25.0213,37.2685,Moon light Furnished Units


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}<dd>and<dd>{Max Temp} °F<dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.head()

,Lat,Lng
2,-21.2000,-175.2000
4,7.4525,126.5842
15,-11.0267,-68.7692
18,4.9533,-77.3660
23,-23.1203,-134.9692


In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))